<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/llama-index-packs/llama-index-packs-multi-tenancy-rag/examples/multi_tenancy_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Tenancy RAG

This notebook shows how to implement Multi-Tenancy RAG with MultiTenancyRAGPack.

### Setup

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR OPENAI API KEY"

### Download data

In [ ]:
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2312.04511.pdf" -O "llm_compiler.pdf"
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2312.06648.pdf" -O "dense_x_retrieval.pdf"

--2024-01-15 17:38:30--  https://arxiv.org/pdf/2312.04511.pdf
Resolving arxiv.org (arxiv.org)... 151.101.67.42, 151.101.195.42, 151.101.131.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 755837 (738K) [application/pdf]
Saving to: 'llm_compiler.pdf'

llm_compiler.pdf    100%[===================>] 738.12K  --.-KB/s    in 0.07s   

2024-01-15 17:38:30 (10.1 MB/s) - 'llm_compiler.pdf' saved [755837/755837]

--2024-01-15 17:38:31--  https://arxiv.org/pdf/2312.06648.pdf
Resolving arxiv.org (arxiv.org)... 151.101.67.42, 151.101.195.42, 151.101.131.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1103758 (1.1M) [application/pdf]
Saving to: 'dense_x_retrieval.pdf'

dense_x_retrieval.p 100%[===================>]   1.05M  --.-KB/s    in 0.1s    

2024-01-15 17:38:31 (8.09 MB/s) - 'dense_x_retrieval.pdf' saved [1103758/1103758]



### Load Data

In [ ]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_files=["dense_x_retrieval.pdf"])
dense_x_retrieval_docs = reader.load_data()

reader = SimpleDirectoryReader(input_files=["llm_compiler.pdf"])
llm_compiler_docs = reader.load_data()

### Download `MultiTenancyRAGPack`

In [ ]:
from llama_index.core.llama_pack import download_llama_pack

In [ ]:
MultiTenancyRAGPack = download_llama_pack(
    "MultiTenancyRAGPack", "./multitenancy_rag_pack"
)

In [ ]:
multitenancy_rag_pack = MultiTenancyRAGPack()

### Add documents for different users

Jerry -> Dense X Retrieval Paper

Ravi -> LLMCompiler Paper

In [ ]:
multitenancy_rag_pack.add(documents=dense_x_retrieval_docs, user="Jerry")
multitenancy_rag_pack.add(documents=llm_compiler_docs, user="Ravi")

### Querying for different users

In [ ]:
# Jerry has Dense X Rerieval paper and should be able to answer following question.
response = multitenancy_rag_pack.run(
    "what are propositions mentioned in the paper?", "Jerry"
)
print(response)

The paper mentions propositions as an alternative retrieval unit choice. Propositions are defined as atomic expressions of meanings in text that correspond to distinct pieces of meaning in the text. They are minimal and cannot be further split into separate propositions. Each proposition is contextualized and self-contained, including all the necessary context from the text to interpret its meaning. The paper demonstrates the concept of propositions using an example about the Leaning Tower of Pisa, where the passage is split into three propositions, each corresponding to a distinct factoid about the tower.


In [ ]:
# Ravi has LLMCompiler
response = multitenancy_rag_pack.run("what are steps involved in LLMCompiler?", "Ravi")
print(response)

LLMCompiler involves three key steps:
1. LLM Planner: This component generates the sequence of different tasks and their dependencies based on user inputs. It identifies the execution flow and defines the function calls with their dependencies.
2. Task Fetching Unit: This component dynamically determines which set of tasks can be executed in parallel. It dispatches the function calls after substituting variables with the actual outputs of the preceding tasks.
3. Executor: This component executes the dispatched function calling tasks using the associated tools. It performs the necessary function calls based on the inputs provided by the Task Fetching Unit.


In [ ]:
# This should not be answered as Jerry does not have information about LLMCompiler
response = multitenancy_rag_pack.run("what are steps involved in LLMCompiler?", "Jerry")
print(response)

I'm sorry, but I couldn't find any information about the steps involved in LLMCompiler in the given context.
